# Biblioteki

In [1]:
import requests
import pandas as pd
from langdetect import detect

##### Wczytanie domyślnych danych o książkach i dropnięcie duplikatów

In [3]:
books_og = pd.read_csv("./books_data/books.csv",low_memory=False)
books = books_og.drop_duplicates(subset="Book-Title")

##### funkcja do pobierania opisu i gatunku książek
sprawdziłem, *genres* może również odnosić się do książek

In [14]:
def get_book_info(title, author, language="en"):

    # language nie zawsze działa rozważyć usunięcie
    google_books_api = f"https://www.googleapis.com/books/v1/volumes?q={title}&lang={language}"

    response = requests.get(google_books_api).json()

    author_good = "NONE"
    description = "NONE"
    genre = "NONE"
    lang = "NONE"

    #iterowanie po różnych ksiąz
    if "items" in response:
        
        response = response["items"]
        
        for item in response:
            
            #to zwróci pusty słownik jakby nie było volume info
            volume_info = item.get("volumeInfo", {})
            authors = volume_info.get("authors", []) if volume_info.get("authors", []) else "NONE"
            
            if author in authors:
                
                #to zwróci listy z opisami, gatunkami, jeśli takie istnieją, i zwróci NONE jeśli nie istnieją
                author_good = author
                description = volume_info.get("description", []) if volume_info.get("description", []) else "NONE"
                genre = volume_info.get("categories", [])        if volume_info.get("categories", [])  else "NONE"
                lang = detect(description)
                break
                
    book_info = pd.DataFrame({'Book-Author':[author_good],
                              'Book-Title': [title],
                              'Book-Description': [description],
                              'Description-Language':[lang],
                              'Book-Genre': [genre]}
                            )

    return book_info

##### faktyczne pobieranie opisów
###### TUTAJ zakomentować .head(x)
nie wiem czy jest co komentować w tej komórce, w miarę przejrzyste rzeczy

In [21]:
books_description_genres = pd.DataFrame(columns=['Book-Author','Book-Title','Book-Description','Description-Language','Book-Genre'])

books_test = books.head(21)

#print(books_test)

for index, row in books_test.iterrows():

    if index%10==0:
        print(index)
        
    title = row['Book-Title']
    author = row['Book-Author']

   
    add_book = get_book_info(title,author)

    
    books_description_genres = pd.concat([books_description_genres,add_book], ignore_index=True)

books_description_genres

0
10
20


,Book-Author,Book-Title,Book-Description,Description-Language,Book-Genre
0,Mark P. O. Morford,Classical Mythology,"Classical Mythology, Sixth Edition, continues ...",en,"[Mythology, Classical]"
1,Richard Bruce Wright,Clara Callan,NONE,vi,[Fiction]
2,Carlo D'Este,Decision in Normandy,The strategy and planning behind D-Day: “The b...,en,[History]
3,Gina Bari Kolata,Flu: The Story of the Great Influenza Pandemic...,NONE,vi,[Epidemics]
4,E. J. W. Barber,The Mummies of Urumchi,"This Study Describes The Remarkable Mummies, T...",en,[Bronze age]
5,Amy Tan,The Kitchen God's Wife,With the same narrative skills and evocative p...,en,[Fiction]
6,Robert Cowley,What If?: The World's Foremost Military Histor...,With its in-depth reflections on the monumenta...,en,[History]
7,Scott Turow,PLEADING GUILTY,The star litigator from a top-notch law firm h...,en,[Fiction]
8,David Cordingly,Under the Black Flag: The Romance and the Real...,Examines the popular image of pirates in moder...,en,[Pirates]
9,Ann Beattie,Where You'll Find Me: And Other Stories,Thirty-six stories--eight appearing in a book ...,en,[Fiction]


In [8]:
books_description_genres.to_csv("./books_data/books_info", index=False)

In [24]:
print(books_description_genres.shape[0])

books_description_genres.drop(books_description_genres[books_description_genres["Book-Author"] == "NONE"].index, inplace =True)
books_description_genres.drop(books_description_genres[books_description_genres["Book-Description"] == "NONE"].index, inplace =True)
books_description_genres.drop(books_description_genres[books_description_genres["Description-Language"] != "en"].index, inplace =True)
books_description_genres.drop(books_description_genres[books_description_genres["Book-Genre"] == "NONE"].index, inplace =True)
print(books_description_genres.shape[0])

21
15
